In [1]:
import pandas as pd
import numpy as np
import findspark
import re
from functools import reduce
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
from matplotlib import pyplot as plt

pd.set_option("display.max_columns", None)

In [2]:
# create entry
spark = SparkSession.builder.appName("MarchMadness").getOrCreate()

def load_dataframe(file_path):
    """read file from relative path."""
    df = spark.read.option('header', True).format('csv').load(file_path)
    return df

### Basic Modeling

In [277]:
season_agg = load_dataframe('./write_data/season_agg.csv').drop('Name')
season_agg.limit(5).toPandas()

,Season,TeamID,Years_D1,GP,PTS,PTSDiff,Pct,FG,%FG,FG3,%FG3,FT,%FT,OR,DR,DR/OR,AST,TO,AST/TO,STL,STL/TO,BLK,PF,HomePTS,AwayPTS,NeutralPTS,HomeDiff,AwayDiff,NeutralDiff,HomePct,AwayPct,NeutralPct,HomeFG,AwayFG,NeutralFG,Home%FG,Away%FG,Neutral%FG,HomeFG3,AwayFG3,NeutralFG3,Home%FG3,Away%FG3,Neutral%FG3,HomeFT,AwayFT,NeutralFT,Home%FT,Away%FT,Neutral%FT,HomeOR,AwayOR,NeutralOR,HomeDR,AwayDR,NeutralDR,HomeDR/OR,AwayDR/OR,NeutralDR/OR,HomeAst,AwayAst,NeutralAst,HomeTO,AwayTO,NeutralTO,HomeSTL,AwaySTL,NeutralSTL,HomeSTL/TO,AwaySTL/TO,NeutralSTL/TO,HomeBlk,AwayBlk,NeutralBlk,HomePF,AwayPF,NeutralPF,rank
0,2003,1119,37.0,25,53.68,-13.8,12.0,41.18,36.51,35.59,10.95,67.61,16.02,7.92,21.56,2.72,11.72,17.36,0.68,5.56,0.32,1.6,19.32,44.0,68.6,40.0,-9.28,-3.8,-0.72,14.29,10.0,0.0,39.91,43.4,36.59,20.19,15.2,1.12,33.19,40.0,26.67,5.89,4.77,0.3,67.69,68.36,54.55,6.56,9.02,0.45,3.64,4.16,0.12,9.08,11.48,1.0,2.49,2.76,8.33,9.29,15.0,13.0,13.0,23.5,17.0,5.71,5.4,5.0,0.44,0.23,0.29,1.07,2.1,4.0,14.93,25.7,17.0,318
1,2003,1308,37.0,28,73.11,5.61,67.86,44.75,35.17,34.68,7.96,73.75,21.69,12.32,23.57,1.91,12.46,13.43,0.93,6.61,0.49,4.04,16.36,56.76,110.13,67.0,2.89,2.36,0.36,64.71,75.0,66.67,43.97,47.61,41.92,21.54,10.21,3.42,31.25,40.43,39.02,4.4,2.78,0.78,74.16,72.65,76.27,10.8,8.7,2.2,5.64,5.71,0.96,11.07,9.71,2.79,1.96,1.7,2.89,10.12,17.13,13.33,9.59,21.75,13.0,7.06,6.25,5.0,0.74,0.29,0.38,3.24,6.5,2.0,11.47,27.88,13.33,92
2,2003,1330,37.0,27,66.63,2.04,44.44,40.22,34.85,31.92,8.78,70.24,21.51,13.96,23.85,1.71,14.22,13.3,1.07,8.26,0.62,2.78,18.33,60.69,77.6,52.0,1.85,0.52,-0.33,50.0,40.0,0.0,40.29,40.72,33.33,19.96,13.9,1.0,30.42,34.74,26.32,4.84,3.67,0.28,68.42,74.67,50.0,11.56,9.34,0.61,7.56,5.96,0.44,13.0,9.93,0.93,1.72,1.66,2.08,14.38,14.3,11.0,11.0,17.1,12.0,9.13,7.7,0.0,0.83,0.45,0.0,3.06,2.3,3.0,15.38,22.6,23.0,187
3,2005,1186,37.0,27,61.41,-6.26,25.93,41.91,35.46,33.27,10.07,66.04,19.0,9.37,21.41,2.28,13.67,16.26,0.84,5.19,0.32,1.7,20.85,35.82,114.0,68.5,-4.96,-1.0,-0.3,23.53,25.0,50.0,41.55,40.67,51.09,22.38,10.25,2.83,32.02,32.59,47.22,6.39,2.65,1.03,61.58,70.2,61.9,7.06,10.37,1.57,4.0,4.7,0.67,8.37,11.78,1.26,2.09,2.5,1.89,8.0,25.13,16.0,8.76,33.13,12.5,5.65,4.38,4.5,0.64,0.13,0.36,1.29,3.0,0.0,11.59,40.75,20.0,276
4,2005,1216,37.0,28,59.46,-10.32,28.57,39.15,38.02,28.79,7.75,62.94,16.22,12.75,21.5,1.69,8.96,14.61,0.61,6.71,0.46,1.93,21.07,39.0,138.2,64.67,-6.79,-2.79,-0.75,35.0,0.0,33.33,39.59,35.16,43.35,26.97,6.55,4.5,30.16,22.73,31.11,5.71,1.2,0.84,70.0,56.61,60.0,7.99,6.43,1.8,5.5,6.11,1.14,9.61,9.39,2.5,1.75,1.54,2.19,5.55,21.2,11.33,8.05,40.2,15.7,6.55,7.4,6.7,0.81,0.18,0.43,1.3,5.0,1.0,12.2,58.2,18.33,297


In [54]:
file_path = '../MarchMadness2021/data/MNCAATourneyCompactResults.csv'
tourney = load_dataframe(file_path).withColumn('Target', col('WScore') - col('LScore'))
tourney.limit(5).toPandas()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Target
0,1985,136,1116,63,1234,54,N,0,9.0
1,1985,136,1120,59,1345,58,N,0,1.0
2,1985,136,1207,68,1250,43,N,0,25.0
3,1985,136,1229,58,1425,55,N,0,3.0
4,1985,136,1242,49,1325,38,N,0,11.0


In [4]:
tourneyseeds = load_dataframe('./data/MNCAATourneySeeds.csv') 
tourneyseeds.limit(5).toPandas()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [7]:
# regular season features with tournament season seeding set up
regular_with_seed = season_agg.select('Season', col('TeamID').alias('WTeamID')
                                      , col('Pct').cast(IntegerType()).alias('WPct')
                                     , col('rank').cast(IntegerType()).alias('WRank')) \
                              .join(tourney, ['Season', 'WTeamID'], 'inner') \
                              .join(tourneyseeds.select('Season', col('Seed').alias('WSeed')
                                                        , col('TeamID').alias('WTeamID')) \
                                    , ['Season', 'WTeamID'], 'inner') \
                              .join(season_agg.select('Season', col('TeamID').alias('LTeamID')
                                                      , col('Pct').cast(IntegerType()).alias('LPct')
                                                      , col('rank').cast(IntegerType()).alias('LRank')) \
                                     , ['Season', 'LTeamID'], 'inner') \
                              .join(tourneyseeds.select('Season', col('Seed').alias('LSeed')
                                                        , col('TeamID').alias('LTeamID'))
                                   , ['Season', 'LTeamID'], 'inner') 
    
#                  .withColumn('LSeed#', concat(tourney.LSeed.substr(2, 2)).cast(IntegerType())) \
#                  .withColumn('WSeed#', concat(tourney.WSeed.substr(2, 2)).cast(IntegerType())) \


regular_with_seed = regular_with_seed \
                 .withColumn('LSeed#', concat(regular_with_seed.LSeed.substr(2, 2)).cast(IntegerType())) \
                 .withColumn('WSeed#', concat(regular_with_seed.WSeed.substr(2, 2)).cast(IntegerType())) \
                    .select('Season', 'LTeamID', 'WTeamID'
                            , col("WScore").cast(IntegerType()), col("LScore").cast(IntegerType())
                            , 'WPct', 'LPct', 'WRank', 'LRank', 'LSeed#', 'WSeed#')
regular_with_seed.limit(5).toPandas()

,Season,LTeamID,WTeamID,WScore,LScore,WPct,LPct,WRank,LRank,LSeed#,WSeed#
0,2015,1459,1116,56,53,76,81,22,80,12,5
1,2004,1150,1403,76,73,68,72,29,34,9,8
2,2006,1393,1401,66,58,72,67,36,26,5,12
3,2007,1172,1268,82,70,75,87,14,46,13,4
4,2010,1196,1140,99,92,84,63,14,48,10,7


In [8]:
# lets make a model based on Pct
regular_with_seed = regular_with_seed \
                     .withColumn('pred_bypct', when(col('WPct') > col('LPct'), 1).otherwise(0)) \
                     .withColumn('pred_byseed', when(col('WSeed#') < col('LSeed#'), 1).otherwise(0)) \
                     .withColumn('pred_byrank', when(col('WRank') < col('LRank'), 1).otherwise(0))

In [9]:
regular_with_seed.select('pred_bypct', 'pred_byseed', 'pred_byrank') \
                .groupby().agg(sum(lit(1)).alias('Actual')
                         ,sum('pred_bypct').alias('PredPct')
                         , sum('pred_byseed').alias('PredSeed')
                         , sum('pred_byrank').alias('PredRank')) \
                .withColumn('PredPct_', round(100*col('PredPct')/col('Actual'), 2)) \
                .withColumn('PredSeed_', round(100*col('PredSeed')/col('Actual'), 2)) \
                .withColumn('PredRank_', round(100*col('PredRank')/col('Actual'), 2)) \
                .show()              

+------+-------+--------+--------+--------+---------+---------+
|Actual|PredPct|PredSeed|PredRank|PredPct_|PredSeed_|PredRank_|
+------+-------+--------+--------+--------+---------+---------+
|  1108|    677|     750|     786|    61.1|    67.69|    70.94|
+------+-------+--------+--------+--------+---------+---------+



* Predicting each match based rank average has better predictability than seed and then winning percentage.
* Average for each match should give us a higher probability than 61%.
* Seed is not available for the first prediction round of submission. 

In [51]:
# lets create the data for training model
# data is since 2003
def map_columns(
    dataframe, original_col, map_win, map_loss, map_win_ex=None, map_loss_ex=None
):
    map_win = [column for column in map_win if column not in map_win_ex]
    map_loss = [column for column in map_loss if column not in map_loss_ex]

    original_col = [
        column
        for column in original_col
        if column not in ["TeamID", "TeamName", "Season"]
    ]
    zip_cols = zip(map_win, map_loss, original_col)
    for map_from, map_to, org_col in zip_cols:
        dataframe = (
            dataframe.withColumn(org_col, col(map_from) - col(map_to))
            .drop(map_from)
            .drop(map_to)
        )

    return dataframe


In [84]:
tourney_lr = tourney.withColumn('Target', col('WScore') - col('LScore')) \
                      .select('Season', 'WTeamID', 'LTeamID', 'Target') \

old_cols = [column for column in season_agg.columns if column != 'Season']
new_cols_L = ['L' + column for column in season_agg.columns if column != 'Season']
new_cols_W = ['W' + column for column in season_agg.columns if column != 'Season']

# rename columns
season_agg_L = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_L[idx])
                   , range(len(old_cols)), season_agg)

season_agg_L = season_agg_L.join(tourney_lr, ['LTeamID', 'Season'], 'inner')

# rename columns
season_agg_W = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_W[idx])
                   , range(len(old_cols)), season_agg)

season_agg_W = season_agg_W.join(tourney_lr, ['WTeamID', 'Season'], 'inner')

season_agg_lr = season_agg_W.join(season_agg_L
                                  , ['WTeamID', 'LTeamID', 'Season', 'Target']
                                  , 'inner')

map_win_ex = ['WTeamID', 'Season', 'Target']
map_loss_ex = ['LTeamID', 'Season', 'Target']
data_target_positive = map_columns(season_agg_lr, old_cols, new_cols_W, new_cols_L
                                   , map_win_ex=map_win_ex, map_loss_ex=map_loss_ex) 

In [85]:
data_target_negative = map_columns(season_agg_lr, old_cols, new_cols_L, new_cols_W
                   , map_win_ex=['LTeamID', 'Season', 'Target']
                   , map_loss_ex=['WTeamID', 'Season', 'Target']) \
                  .drop('WName', 'LName') \
                  .withColumn('Target', -col('Target'))

data = data_target_positive.union(data_target_negative)

In [86]:
# data.withColumn('Target', when(col('Target') < 0, 0).otherwise(1)) \
#     .toPandas().to_csv('./write_data/modeling_data.csv', header=True, index=False, float_format='%.1f')

In [89]:
# training and testing data
model_data = data.withColumn('win_loss', when(col('Target') < 0, 0).otherwise(1)).toPandas()
model_data['Season'] = pd.to_numeric(model_data['Season'])

training = model_data[model_data['Season'] < 2015]
testing = model_data[model_data['Season'] >= 2015]

In [194]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_absolute_error, mean_squared_error


In [147]:
def pipe_score(model, y_label = None, cv=None, scoring=None):
    # lets start with linear regression
    X_train = training.drop(columns=['win_loss', 'Target'])
    y_train = training[y_label].values.ravel()
    imp = SimpleImputer(strategy='mean')
    feat_selector = SelectKBest(f_regression, k=10)
    pipe = Pipeline(steps=[('imp', imp), ('select', feat_selector), ('model', model)])
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring=scoring)
    return scores


In [149]:
# lets start with linear regression
model = LinearRegression()
cv = KFold(n_splits=5)
pipe_score(model, y_label='Target', cv=cv, scoring='neg_mean_absolute_error')

array([ -9.64960871, -10.10377538,  -8.48031173, -10.26516469,
        -9.86017695])

In [239]:
X_train = training.drop(columns=['win_loss', 'Target', 'WTeamID', 'LTeamID', 'Season'])
y_train = training['Target'].values.ravel()

X_test = testing.drop(columns=['win_loss', 'Target', 'WTeamID', 'LTeamID', 'Season'])
y_test = testing['Target'].values.ravel()

model_lr = LinearRegression()
cv = KFold(n_splits=20)
feat_selector = SelectKBest(f_regression, k=10)
pipe_lr = Pipeline(steps=[('imp', imp), ('select', feat_selector), ('model', model_lr)])
pipe_lr.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('imp',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('select',
                 SelectKBest(k=10,
                             score_func=<function f_regression at 0x7fbaf4c0cd90>)),
                ('model',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [240]:
mean_squared_error(y_test, pipe_lr.predict(X_test))**(0.5)

11.364024711667364

In [241]:
#logisitc
y_test = testing['win_loss'].values.ravel()
model_log = LogisticRegression(solver='liblinear', max_iter=1000)
cv = StratifiedKFold(n_splits=5)
feat_selector = SelectKBest(f_regression, k=5)
pipe_log = Pipeline(steps=[('imp', imp), ('select', feat_selector), ('model', model_log)])
pipe_log.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('imp',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('select',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x7fbaf4c0cd90>)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='liblinear', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [242]:
mean_absolute_error(y_test, pipe_log.predict(X_test))

7.4818181818181815

In [220]:
# average MAE of the five folds
-1*scores.mean()

-0.78106392229221

In [120]:
# the average difference in the scores between winning and losing teams in the target data
# prediction average doesn't seem that far off even though it the game can be decided by a single point
y_train[y_train > 0].mean()

11.41388174807198

In [150]:
# lets try simple logisitc regression

model = LogisticRegression(solver='liblinear', max_iter=1000)
cv = StratifiedKFold(n_splits=5)
pipe_score(model, y_label='win_loss', cv=cv, scoring='roc_auc')

array([0.82930638, 0.78010753, 0.72473118, 0.79693962, 0.7742349 ])

In [122]:
# from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [174]:
# submission
df_sub = load_dataframe('./Data/MSampleSubmissionStage1.csv')
df_sub = df_sub \
                .withColumn('Season', split(df_sub.ID, '_').getItem(0)) \
                .withColumn('WTeamID', split(df_sub.ID, '_').getItem(1)) \
                .withColumn('LTeamID', split(df_sub.ID, '_').getItem(2))

df_sub.limit(5).toPandas()

,ID,Pred,Season,WTeamID,LTeamID
0,2015_1107_1112,0.5,2015,1107,1112
1,2015_1107_1116,0.5,2015,1107,1116
2,2015_1107_1124,0.5,2015,1107,1124
3,2015_1107_1125,0.5,2015,1107,1125
4,2015_1107_1129,0.5,2015,1107,1129


In [271]:
df_sub.count()

11390

In [181]:
old_cols = [column for column in season_agg.columns if column != 'Season']

new_cols_L = ['L' + column for column in season_agg.columns if column != 'Season']
new_cols_W = ['W' + column for column in season_agg.columns if column != 'Season']

# rename columns
season_agg_L = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_L[idx])
                   , range(len(old_cols)), season_agg)

season_agg_L = season_agg_L.join(df_sub.drop('Pred'), ['LTeamID', 'Season'], 'inner')

# rename columns
season_agg_W = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_W[idx])
                   , range(len(old_cols)), season_agg)

season_agg_W = season_agg_W.join(df_sub.drop('ID', 'Pred'), ['WTeamID', 'Season'], 'inner')

season_agg_lr = season_agg_W.join(season_agg_L
                                  , ['WTeamID', 'LTeamID', 'Season']
                                  , 'inner')

map_win_ex = ['WTeamID', 'Season']
map_loss_ex = ['LTeamID', 'Season']
data_test = map_columns(season_agg_lr, old_cols, new_cols_W, new_cols_L
                                   , map_win_ex=map_win_ex, map_loss_ex=map_loss_ex) 

In [183]:
data_test = data_test.drop('WTeamID', 'LTeamID').toPandas()
data_test.head()

,Season,ID,Years_D1,GP,PTS,PTSDiff,Pct,FG,%FG,FG3,%FG3,FT,%FT,OR,DR,DR/OR,AST,TO,AST/TO,STL,STL/TO,BLK,PF,HomePTS,AwayPTS,NeutralPTS,HomeDiff,AwayDiff,NeutralDiff,HomePct,AwayPct,NeutralPct,HomeFG,AwayFG,NeutralFG,Home%FG,Away%FG,Neutral%FG,HomeFG3,AwayFG3,NeutralFG3,Home%FG3,Away%FG3,Neutral%FG3,HomeFT,AwayFT,NeutralFT,Home%FT,Away%FT,Neutral%FT,HomeOR,AwayOR,NeutralOR,HomeDR,AwayDR,NeutralDR,HomeDR/OR,AwayDR/OR,NeutralDR/OR,HomeAst,AwayAst,NeutralAst,HomeTO,AwayTO,NeutralTO,HomeSTL,AwaySTL,NeutralSTL,HomeSTL/TO,AwaySTL/TO,NeutralSTL/TO,HomeBlk,AwayBlk,NeutralBlk,HomePF,AwayPF,NeutralPF,rank
0,2018,2018_1112_1347,0.0,2.0,13.94,7.65,16.91,8.09,0.86,2.56,-2.89,2.70,1.18,-0.78,3.62,0.52,3.36,0.29,0.25,-1.34,-0.13,1.65,0.75,31.79,-19.55,8.50,8.01,0.40,-0.76,35.96,11.11,-42.86,10.04,7.15,5.16,-1.54,-0.27,2.68,6.79,2.98,-7.40,-1.93,-1.11,0.15,5.06,2.38,-2.74,2.45,-2.32,1.05,0.28,-1.84,0.78,4.71,-3.72,2.63,1.01,0.07,0.46,7.09,-3.89,3.93,3.72,-6.67,0.1,-1.41,-1.11,-1.3,-0.47,0.04,-0.13,2.04,1.00,1.61,5.89,-11.33,3.04,-148.0
1,2015,2015_1112_1374,0.0,2.0,7.03,8.13,9.93,0.69,-0.58,-0.02,0.54,-0.55,0.64,-0.40,1.25,0.20,-1.35,-1.23,0.02,0.40,0.09,-0.44,1.63,2.34,24.80,3.00,4.47,2.12,1.54,6.05,20.00,0.00,-0.18,2.63,2.63,-0.55,-2.51,2.47,-1.75,0.21,6.88,0.22,-0.55,0.87,-4.95,4.23,4.94,-2.17,1.38,1.42,0.13,-1.12,0.57,0.51,-1.01,1.75,0.03,0.40,0.27,-0.94,-2.10,0.17,-1.32,-0.32,1.0,0.08,0.65,1.3,0.09,0.05,0.07,-1.16,0.80,0.33,0.40,5.65,3.34,-19.0
2,2016,2016_1112_1462,0.0,1.0,-0.13,1.90,-8.62,2.98,0.90,2.10,-1.32,-0.79,-0.49,-1.42,0.67,0.32,-1.86,0.07,-0.15,-2.40,-0.19,1.73,-1.98,-6.17,37.61,-7.00,5.63,-1.64,-2.10,-6.06,-5.56,-20.00,4.59,0.74,-0.78,4.98,-3.40,-0.69,0.72,7.60,-2.84,0.18,-0.89,-0.61,-2.37,-2.09,8.11,0.90,-1.59,0.22,-0.40,-0.73,-0.29,1.94,-0.86,-0.41,0.48,0.21,0.04,-3.06,3.56,-4.00,-0.42,2.95,3.4,-2.84,-1.83,-2.0,-0.26,-0.15,-0.24,0.23,6.77,1.80,-3.37,5.61,0.20,5.0
3,2018,2018_1120_1138,7.0,-1.0,-1.32,1.88,2.37,-3.48,-3.71,-0.57,0.31,8.98,7.10,0.10,-0.79,-0.08,-2.32,-0.14,-0.17,1.23,0.11,1.08,-1.27,6.91,-12.97,-4.00,3.35,-0.77,-0.69,6.73,-1.39,-6.67,-3.56,-6.17,-0.08,-0.10,-2.25,-1.35,-0.84,3.53,-7.46,0.97,0.11,-0.76,8.24,13.58,1.12,3.65,2.94,0.50,0.63,0.21,-0.74,1.40,-1.79,-0.39,-0.01,-0.50,0.50,-0.38,-6.94,-0.93,0.43,-2.81,3.4,1.67,-0.06,1.6,0.14,0.06,-0.02,1.40,0.60,1.17,-0.73,-1.11,-0.40,-47.0
4,2018,2018_1120_1301,0.0,0.0,2.16,3.31,12.50,-3.11,-3.52,-0.60,1.99,8.61,5.04,-0.06,2.53,0.22,-1.72,0.06,-0.15,-0.06,-0.01,1.60,1.72,-4.09,15.11,3.25,0.44,2.41,0.47,2.76,30.36,35.00,-2.42,-5.91,-0.70,-4.29,-0.13,0.91,1.14,-1.92,-5.35,1.27,0.50,0.24,9.16,8.78,6.44,-0.32,4.32,1.03,-1.34,1.10,0.18,-1.19,2.60,1.13,0.28,0.15,0.42,-2.16,-2.21,0.90,0.27,-1.80,-0.1,-0.16,-0.50,1.6,-0.04,0.02,0.12,0.98,2.24,3.00,-0.08,4.86,0.35,-27.0


In [274]:
df_sub.select('ID').join(spark.createDataFrame(data_test[['ID']]), 'ID', 'left_anti').toPandas()

,ID
0,2015_1246_1459
1,2015_1231_1246
2,2015_1246_1437
3,2015_1138_1246
4,2015_1246_1314
...,...
62,2015_1246_1385
63,2015_1246_1301
64,2015_1153_1246
65,2015_1246_1332


In [279]:
season_agg.filter(col('Season') == 2015).filter(col('TeamID') == 1459).toPandas()

,Season,TeamID,Years_D1,GP,PTS,PTSDiff,Pct,FG,%FG,FG3,%FG3,FT,%FT,OR,DR,DR/OR,AST,TO,AST/TO,STL,STL/TO,BLK,PF,HomePTS,AwayPTS,NeutralPTS,HomeDiff,AwayDiff,NeutralDiff,HomePct,AwayPct,NeutralPct,HomeFG,AwayFG,NeutralFG,Home%FG,Away%FG,Neutral%FG,HomeFG3,AwayFG3,NeutralFG3,Home%FG3,Away%FG3,Neutral%FG3,HomeFT,AwayFT,NeutralFT,Home%FT,Away%FT,Neutral%FT,HomeOR,AwayOR,NeutralOR,HomeDR,AwayDR,NeutralDR,HomeDR/OR,AwayDR/OR,NeutralDR/OR,HomeAst,AwayAst,NeutralAst,HomeTO,AwayTO,NeutralTO,HomeSTL,AwaySTL,NeutralSTL,HomeSTL/TO,AwaySTL/TO,NeutralSTL/TO,HomeBlk,AwayBlk,NeutralBlk,HomePF,AwayPF,NeutralPF,rank
0,2015,1459,26.0,32,65.88,5.81,81.25,45.43,35.15,37.13,9.44,68.98,20.26,9.19,22.81,2.48,13.13,10.75,1.22,6.41,0.6,2.56,17.03,51.31,85.55,69.2,2.09,2.22,1.5,68.75,90.91,100.0,45.23,44.14,49.16,17.98,11.62,5.55,36.27,36.05,42.5,4.89,2.94,1.61,71.22,67.0,70.27,6.93,9.63,3.7,3.84,3.94,1.41,8.94,10.13,3.75,2.33,2.57,2.67,11.06,16.55,12.2,7.0,15.64,12.0,6.13,7.27,5.4,0.88,0.47,0.45,2.25,2.82,3.0,12.56,23.82,16.4,80


In [281]:
file_path = '../MarchMadness2021/data/MRegularSeasonCompactResults.csv'
compact = load_dataframe(file_path)
compact.filter(col('Season') == 2015).filter(col('WTeamID') == 1246).toPandas()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2015,11,1246,85,1213,45,H,0
1,2015,13,1246,71,1138,52,H,0
2,2015,15,1246,72,1242,40,N,0
3,2015,18,1246,89,1131,65,H,0
4,2015,20,1246,86,1286,28,H,0
5,2015,22,1246,92,1426,44,H,0
6,2015,27,1246,58,1344,38,H,0
7,2015,32,1246,63,1400,51,H,0
8,2015,34,1246,82,1184,49,H,0
9,2015,37,1246,56,1162,46,H,0


In [256]:
#linear reg
data_lr = data_test[['ID']].copy()
data_lr['pred'] = pipe_lr.predict(data_test.drop(columns=['Season', 'ID']))
data_lr.head()

,ID,pred
0,2018_1112_1347,19.857279
1,2015_1112_1374,6.279234
2,2016_1112_1462,-0.519242
3,2018_1120_1138,7.282465
4,2018_1120_1301,3.432761


In [269]:
df_sub.count()

11390

In [265]:
data_lr.to_csv('./write_data/01_spread_pred.csv', header=True, index=False, float_format='%.1f')

In [259]:
data_test.shape, data_log.shape

((11323, 78), (11323, 1))

In [264]:
#logistic regression
data_log = data_test[['ID']].copy()
pipe_log.predict_log_proba(data_test.drop(columns=['Season', 'ID']))[:, 1]
# data_log.head()

array([-6.46671826, -5.93132364, -6.48236503, ..., -5.54641559,
       -5.92434612, -6.64446495])